In [1]:
# --- Environment & imports ---
from dotenv import load_dotenv
import os
import re
import json
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from langdetect import detect, DetectorFactory
from openai import OpenAI

# Load .env file (THIS is the missing step)
load_dotenv()

# Sanity check (fail fast)
assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not found. Check your .env file."

# Init OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Reproducibility
DetectorFactory.seed = 42
tqdm.pandas()

# Paths
DATA_DIR = Path("..") / "data" / "userstudy"
DATA_DIR


PosixPath('../data/userstudy')

In [3]:
INPUT_RAW_CSV = DATA_DIR / "UserStudy - Responses.csv"

df = pd.read_csv(INPUT_RAW_CSV)
assert "free_text" in df.columns, "CSV must contain column 'free_text'"

df["free_text"] = df["free_text"].astype(str).str.strip()
df.loc[df["free_text"].isin(["", "nan", "None"]), "free_text"] = np.nan

OUT0_CSV  = DATA_DIR / "step0_raw_cleaned.csv"
OUT0_XLSX = DATA_DIR / "step0_raw_cleaned.xlsx"

df.to_csv(OUT0_CSV, index=False)
df.to_excel(OUT0_XLSX, index=False)

print("Saved:", OUT0_CSV.name, "and", OUT0_XLSX.name)


Saved: step0_raw_cleaned.csv and step0_raw_cleaned.xlsx


In [4]:
def safe_detect_lang(text: str) -> str:
    if text is None or not isinstance(text, str):
        return "unknown"
    t = text.strip()
    if not t:
        return "unknown"
    try:
        return detect(t)
    except:
        return "unknown"


PROMPT_DETECT_SYSTEM = "You are a strict classifier for user-study responses."

PROMPT_DETECT_USER_TEMPLATE = """Decide if the text is a usable user prompt for map generalization.

Return ONLY one token:
- PROMPT
- NOT_PROMPT

Text:
{TEXT}
"""

def looks_like_prompt_llm(text: str, model="gpt-4o-mini") -> bool:
    if text is None or not isinstance(text, str) or not text.strip():
        return False
    msg = PROMPT_DETECT_USER_TEMPLATE.format(TEXT=text.strip())
    r = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role":"system", "content": PROMPT_DETECT_SYSTEM},
            {"role":"user", "content": msg},
        ],
    )
    return r.choices[0].message.content.strip() == "PROMPT"


In [5]:
IN1_XLSX = DATA_DIR / "step0_raw_cleaned.xlsx"
df = pd.read_excel(IN1_XLSX)

assert "free_text" in df.columns

df["step1_is_prompt"] = df["free_text"].progress_apply(looks_like_prompt_llm)
df["step1_text"] = df["free_text"].where(df["step1_is_prompt"], np.nan)

# audit drop reason
df["drop_reason"] = np.nan
df.loc[df["free_text"].isna(), "drop_reason"] = "empty_free_text"
df.loc[df["free_text"].notna() & (~df["step1_is_prompt"]), "drop_reason"] = "not_prompt_like"

OUT1_CSV  = DATA_DIR / "step1_prompt_detected.csv"
OUT1_XLSX = DATA_DIR / "step1_prompt_detected.xlsx"
df.to_csv(OUT1_CSV, index=False)
df.to_excel(OUT1_XLSX, index=False)

print("Saved:", OUT1_CSV.name, "and", OUT1_XLSX.name)


100%|██████████| 786/786 [07:45<00:00,  1.69it/s]

Saved: step1_prompt_detected.csv and step1_prompt_detected.xlsx



/var/folders/6r/lw1cp6x117zfdkxgl862fymc0000gn/T/ipykernel_99220/561570398.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'empty_free_text' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df["free_text"].isna(), "drop_reason"] = "empty_free_text"


In [6]:
TRANSLATE_SYSTEM = "You translate text to English with high fidelity."

TRANSLATE_USER_TEMPLATE = """Translate the following text into English.

Rules:
- Preserve meaning and tone.
- Keep it as a prompt/command.
- Do NOT add new information.
- Output a single line only.

Text:
{TEXT}
"""

def translate_to_english(text: str, model="gpt-4o-mini") -> str:
    if text is None or not isinstance(text, str) or not text.strip():
        return np.nan
    msg = TRANSLATE_USER_TEMPLATE.format(TEXT=text.strip())
    r = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role":"system", "content": TRANSLATE_SYSTEM},
            {"role":"user", "content": msg},
        ],
    )
    return r.choices[0].message.content.strip()


IN2_XLSX = DATA_DIR / "step1_prompt_detected.xlsx"
df = pd.read_excel(IN2_XLSX)

df["step1_lang"] = df["step1_text"].apply(lambda x: safe_detect_lang(x) if isinstance(x, str) else "unknown")

need_translate = df["step1_is_prompt"] & (df["step1_lang"] != "en") & df["step1_text"].notna()

df["step2_text"] = np.nan
df.loc[df["step1_is_prompt"] & (df["step1_lang"] == "en"), "step2_text"] = df.loc[
    df["step1_is_prompt"] & (df["step1_lang"] == "en"),
    "step1_text"
]

df.loc[need_translate, "step2_text"] = df.loc[need_translate, "step1_text"].progress_apply(translate_to_english)

OUT2_CSV  = DATA_DIR / "step2_translated.csv"
OUT2_XLSX = DATA_DIR / "step2_translated.xlsx"
df.to_csv(OUT2_CSV, index=False)
df.to_excel(OUT2_XLSX, index=False)

print("Saved:", OUT2_CSV.name, "and", OUT2_XLSX.name)


/var/folders/6r/lw1cp6x117zfdkxgl862fymc0000gn/T/ipykernel_99220/4112793507.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['I want to use the map on the right side because some unnecessary details have been removed.'
 'I want to use the map on the right side because the buildings under 100 square feet have been removed.'
 'I want to use the map on the right side because unnecessary details have been removed.'
 'I want to use the map on the left side because on the right side, the buildings are not properly separated.'
 'I want to use the map on the right because buildings under 100 square meters have been removed.'
 'merge neighbouring building polygons by removing small gaps between them'
 'remove small buildings, simplify the building outlines, and reduce detail by eliminating minor shapes.'
 'remove very small buildings, simplify building outlines, and make remaining shapes more rectangular and

Saved: step2_translated.csv and step2_translated.xlsx


In [7]:
GRAMMAR_SYSTEM = "You perform minimal grammatical correction without changing tone or meaning."

GRAMMAR_USER_TEMPLATE = """Correct grammar, punctuation, spacing, and number formatting in the sentence.

Rules:
- Do NOT change meaning.
- Do NOT change tone.
- Do NOT paraphrase.
- Do NOT add/remove content unless grammatically required.
- Keep it as ONE sentence prompt.
- Output ONE line only.

Text:
{TEXT}
"""

def minimal_grammar_fix(text: str, model="gpt-4o-mini") -> str:
    if text is None or not isinstance(text, str) or not text.strip():
        return np.nan
    msg = GRAMMAR_USER_TEMPLATE.format(TEXT=text.strip())
    r = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role":"system", "content": GRAMMAR_SYSTEM},
            {"role":"user", "content": msg},
        ],
    )
    return r.choices[0].message.content.strip()


IN3_XLSX = DATA_DIR / "step2_translated.xlsx"
df = pd.read_excel(IN3_XLSX)

df["step3_text"] = df["step2_text"].progress_apply(minimal_grammar_fix)
df["cleaned_text"] = df["step3_text"]

OUT3_CSV  = DATA_DIR / "step3_cleaned.csv"
OUT3_XLSX = DATA_DIR / "step3_cleaned.xlsx"
df.to_csv(OUT3_CSV, index=False)
df.to_excel(OUT3_XLSX, index=False)

print("Saved:", OUT3_CSV.name, "and", OUT3_XLSX.name)


100%|██████████| 786/786 [07:54<00:00,  1.66it/s]

Saved: step3_cleaned.csv and step3_cleaned.xlsx


In [11]:
TH_EXIST_SYSTEM = "You are a strict binary classifier for map-generalization prompts."

# IMPORTANT: escape JSON braces with {{ and }} because we use .format()
TH_EXIST_USER = """Determine whether the prompt contains a threshold/constraint condition (explicit or implicit).

Return ONLY a JSON object with exactly these keys:
{{
  "threshold_exist": true/false,
  "evidence": "short phrase indicating the threshold, or empty string if none"
}}

Prompt:
{TEXT}
"""

def detect_threshold_llm(text: str, model="gpt-4o-mini"):
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return {"threshold_exist": False, "evidence": ""}

    t = str(text).strip()
    if not t:
        return {"threshold_exist": False, "evidence": ""}

    r = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": TH_EXIST_SYSTEM},
            {"role": "user", "content": TH_EXIST_USER.format(TEXT=t)},
        ],
        response_format={"type": "json_object"},
    )

    try:
        obj = json.loads(r.choices[0].message.content)
    except Exception:
        return {"threshold_exist": False, "evidence": ""}

    # normalize keys defensively
    obj = {str(k).strip().replace('"', ""): v for k, v in (obj or {}).items()}

    return {
        "threshold_exist": bool(obj.get("threshold_exist", False)),
        "evidence": "" if obj.get("evidence", "") is None else str(obj.get("evidence", "")).strip(),
    }


In [12]:
IN4_XLSX = DATA_DIR / "step3_cleaned.xlsx"
df = pd.read_excel(IN4_XLSX)

# Apply only on non-empty cleaned_text to save API calls
mask = df["cleaned_text"].notna()

results = pd.Series([{"threshold_exist": False, "evidence": ""}] * len(df), index=df.index)
results.loc[mask] = df.loc[mask, "cleaned_text"].progress_apply(detect_threshold_llm)

df["threshold_exist"] = results.apply(lambda x: x["threshold_exist"])
df["threshold_evidence"] = results.apply(lambda x: x["evidence"])

OUT4_CSV  = DATA_DIR / "step4_threshold_exist.csv"
OUT4_XLSX = DATA_DIR / "step4_threshold_exist.xlsx"
df.to_csv(OUT4_CSV, index=False)
df.to_excel(OUT4_XLSX, index=False)

print("Saved:", OUT4_CSV.name, "and", OUT4_XLSX.name)


100%|██████████| 617/617 [08:39<00:00,  1.19it/s]

Saved: step4_threshold_exist.csv and step4_threshold_exist.xlsx


In [15]:
TH_KNOWN_SYSTEM = "You are a strict classifier for threshold specification in map-generalization prompts."

# IMPORTANT: escape JSON braces because we use .format()
TH_KNOWN_USER = """The prompt below contains a threshold/constraint.

Decide whether the threshold is KNOWN or UNKNOWN.

KNOWN:
- numeric threshold (e.g., 10 meters, 200 m², 5 buildings)
- qualitative but explicit threshold (e.g., very small buildings, closely spaced, short distance)

UNKNOWN:
- placeholder/vague threshold (e.g., X meters, N, some distance, a threshold, less than the threshold)

Return ONLY a JSON object with exactly these keys:
{{
  "threshold_known": true/false,
  "evidence": "If threshold_known is true: copy the short phrase that specifies the threshold. Otherwise: empty string."
}}

Prompt:
{TEXT}
"""

def threshold_known_llm(text: str, model="gpt-4o-mini"):
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return {"threshold_known": False, "evidence": ""}

    t = str(text).strip()
    if not t:
        return {"threshold_known": False, "evidence": ""}

    r = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": TH_KNOWN_SYSTEM},
            {"role": "user", "content": TH_KNOWN_USER.format(TEXT=t)},
        ],
        response_format={"type": "json_object"},
    )

    try:
        raw = json.loads(r.choices[0].message.content)
    except Exception:
        return {"threshold_known": False, "evidence": ""}

    # normalize keys (defensive)
    raw = {str(k).strip().replace('"', ""): v for k, v in (raw or {}).items()}

    known = bool(raw.get("threshold_known", False))
    evidence = raw.get("evidence", "")
    evidence = "" if evidence is None else str(evidence).strip()

    # enforce your rule
    if not known:
        evidence = ""

    return {"threshold_known": known, "evidence": evidence}


In [16]:
IN5_XLSX = DATA_DIR / "step4_threshold_exist.xlsx"
df = pd.read_excel(IN5_XLSX)

# create columns
if "threshold_known" not in df.columns:
    df["threshold_known"] = pd.NA
if "threshold_known_evidence" not in df.columns:
    df["threshold_known_evidence"] = ""

mask = (df["threshold_exist"] == True) & df["cleaned_text"].notna()

results = df.loc[mask, "cleaned_text"].progress_apply(threshold_known_llm)

df.loc[mask, "threshold_known"] = results.apply(lambda x: x["threshold_known"]).values
df.loc[mask, "threshold_known_evidence"] = results.apply(lambda x: x["evidence"]).values

# enforce your rule: if threshold_exist True, threshold_known must be True/False
mask_bad = (df["threshold_exist"] == True) & (df["threshold_known"].isna())
df.loc[mask_bad, "threshold_known"] = False
df.loc[mask_bad, "threshold_known_evidence"] = ""

OUT5_CSV  = DATA_DIR / "step5_threshold_known.csv"
OUT5_XLSX = DATA_DIR / "step5_threshold_known.xlsx"
df.to_csv(OUT5_CSV, index=False)
df.to_excel(OUT5_XLSX, index=False)

print("Saved:", OUT5_CSV.name, "and", OUT5_XLSX.name)


100%|██████████| 434/434 [04:48<00:00,  1.50it/s]

Saved: step5_threshold_known.csv and step5_threshold_known.xlsx
